In [ ]:
import numpy as np
import pandas as pd
from datapipeline import DataPipeline
from clusterpipeline import ClusterPipeline
from portfoliopipeline import PortfolioPipeline
from backtestpipeline import BacktestPipeline

%autoreload 2

# current idea now is cluster --> use johansen test to test for cointegration --> minimise portmanteau stat --> parametric threshold
# or cluster --> engle granger test --> pairs with kalman filter --> parametric threshold

# maybe use daily data to train

data_pipe = DataPipeline([], (2016, 1, 1), (2020, 1, 1), (2022, 1, 1), (2022, 12, 31))
cluster_pipe = ClusterPipeline() 
portfolio_pipe = PortfolioPipeline()
backtest_pipe = BacktestPipeline()


In [ ]:
training_df, validation_df, testing_df, training_and_validation_df = data_pipe.preprocess_and_split_data()

In [ ]:
display('##### Finding clusters now #####')
clusters = [training_df.columns] if training_df.shape[1] < 15 else cluster_pipe.find_clusters(training_df)

In [ ]:
# get pairs for validation test
validation_pairs = portfolio_pipe.find_pairs_from_clusters(training_df, clusters)
# if validation_pairs is None or validation_pairs.shape[0] == 0:
# 	display(f'No validated pairs')
# 	return

initial_capital = 5000

# validation backtest
validation_backtest_results = backtest_pipe.validation_backtest(training_df, validation_df, validation_pairs, initial_capital=initial_capital)

for pair_key in validation_backtest_results:
	backtest_pipe.plot_pair_backtest(validation_backtest_results, pair_key)



# filter results
filtered_validation_backtest_results = {}

for pair_key in validation_backtest_results:
	margin = validation_backtest_results[pair_key]['margin']
	if (margin[-1]-margin[0])/margin[0] >= 0.1:
		filtered_validation_backtest_results[pair_key] = validation_backtest_results[pair_key]

num_successful = len(filtered_validation_backtest_results)
num_total = len(validation_backtest_results)
display(f'{num_successful}/{num_total} = {num_successful/num_total*100:.2f}% have +PnL')
display(filtered_validation_backtest_results)

In [ ]:

# # if len(filtered_validation_pairs) == 0:
# # 	display(f'No positive validated pairs')
# # 	return


# # get final test pairs
# test_pairs = portfolio_pipe.revalidate_pairs(filtered_validation_backtest_results)



# # test backtest
# test_backtest_results = backtest_pipe.test_backtest(training_and_validation_df, testing_df, test_pairs, filtered_validation_backtest_results, initial_capital=initial_capital)
# for pair_key in test_backtest_results:
# 	backtest_pipe.plot_pair_backtest(test_backtest_results, pair_key)


# display(test_pairs)
# display(f'Total pairs validated / total pairs found = {num_successful}/{num_total} = {num_successful/num_total*100:.2f}% pairs ')